## Passo 1: Calcolare la media dei tempi di esecuzione
Il seguente script è necessario per calcolare la media delle rilevazioni dei tempi di esecuzione.

In [27]:
import pandas as pd

# Leggi il file CSV
df = pd.read_csv('output/analytics_data.csv')

# Raggruppa i dati per le colonne 0, 1 e 2 e calcola la media sulla colonna 3
df_grouped = df.groupby([df.columns[0], df.columns[1], df.columns[2]])[df.columns[3]].mean().reset_index()

# Salva il risultato in un nuovo file CSV
df_grouped.to_csv('output/analytics_data_mean.csv', index=False)


## Passo 2: Calcolo di Speed-up ed Efficienza
Lo script seguente calcola e aggiunge le colonne di Speed-up ed Efficienza al file .CSV

In [28]:
import numpy as np
import pandas as pd

df = pd.read_csv('output/analytics_data_mean.csv')


def calculate_speedup(row):
    base_row = df[(df['NPROC'] == 1) & (df['NINPUT'] == row['NINPUT']) & (df['STRATEGIA'] == row['STRATEGIA'])]
    if len(base_row) > 0:
        return base_row['TEMPO'].values[0] / row['TEMPO']
    return np.nan


df['SPEEDUP'] = df.apply(calculate_speedup, axis=1)

# Calcola l'efficienza solo per i valori di speedup diversi da NaN
df['EFFICIENZA'] = np.where(df['SPEEDUP'].notna(), df['SPEEDUP'] / df['NPROC'], np.nan)

df.to_csv('output/analytics_data_complete.csv', index=False)


## Passo 3: Disegnare i grafici
Il seguente script si occupa della lettura del file .CSV per estrarre i dati di nostro interesse e disegnare i grafici.

In [20]:
import pandas as pd
import matplotlib.pyplot as plt

def create_graphs(dataframe, x_col, y_cols, nproc_fixed=None, ninput_fixed=None, x_min=None, x_max=None):
    """    
    Args:
        dataframe (pd.DataFrame): Il DataFrame contenente i dati.
        x_col (str): Nome della colonna da utilizzare per le ascisse.
        y_cols (list): Lista di nomi delle colonne da utilizzare per le ordinate.
        nproc_fixed (int): Numero di processori fisso (usato solo se specificato).
        ninput_fixed (int): Numero di input fisso (usato solo se specificato).
        save_path (str): Percorso per salvare i grafici (formato: 'nome_file.png').
        x_min (float): Valore minimo per le ascisse.
        x_max (float): Valore massimo per le ascisse.
    """
    
    if x_min is not None and x_max is not None:
        dataframe = dataframe[(dataframe[x_col] >= x_min) & (dataframe[x_col] <= x_max)]

    for y_col in y_cols:
        for strategy in dataframe['STRATEGIA'].unique():
            if nproc_fixed is not None and ninput_fixed is None:
                data = dataframe[(dataframe['NPROC'] == nproc_fixed) & (dataframe['STRATEGIA'] == strategy)]
                x_data = data[x_col]
            else:
                data = dataframe[(dataframe['NINPUT'] == ninput_fixed) & (dataframe['STRATEGIA'] == strategy)]
                x_data = data['NPROC']
            
            plt.plot(x_data, data[y_col], label=f'Strategia {strategy}')
        
        plt.xlabel(x_col)
        plt.ylabel(y_col)
        if nproc_fixed is not None and ninput_fixed is None:
            plt.xticks(dataframe['NINPUT'].unique())  # Imposta le ascisse con valori specifici
            title = f'NPROC {nproc_fixed}, {y_col} vs {x_col}'
        else:
            plt.xticks(dataframe['NPROC'].unique())  # Imposta le ascisse con valori specifici
            title = f'NINPUT {ninput_fixed}, {y_col} vs {x_col}'
        
        plt.legend()
        plt.title(title)
        if nproc_fixed is not None and ninput_fixed is None:
            plt.savefig(f'output/plots/nproc_fixed_{y_col.lower()}.png')
        else:
            plt.savefig(f'output/plots/ninput_fixed_{y_col.lower()}.png')
        plt.close()

# Esempio di utilizzo con intervallo per le ascisse
data = pd.read_csv('output/analytics_data_complete.csv')
create_graphs(data, 'NINPUT', ['TEMPO', 'SPEEDUP', 'EFFICIENZA'], nproc_fixed=8, x_min=100000, x_max=500000000)
create_graphs(data, 'NPROC', ['TEMPO', 'SPEEDUP', 'EFFICIENZA'], ninput_fixed=200000000, x_min=8, x_max=64)
